In [ ]:
#Input
filtered_count_data = '../nf_output/data/Visium_FFPE_V43T08-041_A_filtered.csv'
spatial_data = '../nf_output/data/Visium_FFPE_V43T08-041_A_annotation.csv'
sample_name = 'Visium_FFPE_V43T08-041_A'
path_to_project = '~/Project'

In [ ]:
import torch
import re
import os
import sys
print(path_to_project)
sys.path.append(os.path.join(path_to_project, 'Spatial-Transcriptomics/PDAC/clustering/GraphST_clustering'))
from graphST_clustering_functions import *
from GraphST.GraphST import GraphST

In [8]:
#Output
sample = os.path.basename(filtered_count_data)
parts = sample.split('_')
last_char = parts[-1].split('.')[0]
if last_char.isdigit() == True: #simulation
    seed = last_char
    graphST_output = f"{sample_name}_sim_{seed}_clustered.h5ad"
else:
    graphST_output = f"{sample_name}_ref_clustered.h5ad"

In [9]:
# Run device, by default, the package is implemented on 'cpu'. We recommend using GPU.
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# the location of R, which is necessary for mclust algorithm. Please replace the path below with local R installation path
os.environ["R_HOME"] = "/Library/Frameworks/R.framework/Resources"

In [10]:
data, col_label = data_preprocessing(filtered_count_data, spatial_data)

In [11]:
model = GraphST.GraphST(data, device=device)
# train model
data = model.train()

Begin to train ST data...


100%|██████████| 600/600 [04:24<00:00,  2.27it/s]


Optimization finished for ST data!


In [12]:
# clustering 
methods = ['mclust','louvain', 'leiden']
data = graphST_methods_loop(data, methods, list(range(2,11)), refinement= True)

R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%
Searching resolution...
... for [2, 3, 4, 5, 6, 7, 8, 9, 10]


/Users/memarchand/Project/Spatial-Transcriptomics/PDAC/clustering/envrt_clustering/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


resolution=1.49, cluster number=15
resolution=1.48, cluster number=15
resolution=1.47, cluster number=16
resolution=1.46, cluster number=16
resolution=1.45, cluster number=16
resolution=1.44, cluster number=16
resolution=1.43, cluster number=16
resolution=1.42, cluster number=16
resolution=1.4100000000000001, cluster number=15
resolution=1.4000000000000001, cluster number=15
resolution=1.3900000000000001, cluster number=15
resolution=1.3800000000000001, cluster number=15
resolution=1.37, cluster number=15
resolution=1.36, cluster number=15
resolution=1.35, cluster number=15
resolution=1.34, cluster number=15
resolution=1.33, cluster number=15
resolution=1.32, cluster number=15
resolution=1.31, cluster number=15
resolution=1.3, cluster number=15
resolution=1.29, cluster number=15
resolution=1.28, cluster number=15
resolution=1.27, cluster number=15
resolution=1.26, cluster number=15
resolution=1.25, cluster number=15
resolution=1.24, cluster number=15
resolution=1.23, cluster number=15


In [13]:
data.write_h5ad(graphST_output)